In [2]:
import sympy
import warnings 
import polars as pl
warnings.filterwarnings('ignore')   
import pandas as pd
import numpy as np 
from draw import * 
import os
from tqdm import tqdm
import pickle

code = 'RB'
suffix = f'{code}99_1m'
part = '20250426_100441_OMBx2l'
model_pth =  rf'./GA_Shane/outputs/{part}/hall_of_fame.csv'
project_pth = rf'./GA_Shane/outputs/{part}/'

print(f"Your axis file is in {part}")
n_bar = 1
freq = 60
i = -1

"""
Calculate factor based on a generated equation 
"""
if n_bar == 1:
    csv_path =   rf"~/quant/dragon/{suffix}.csv"
    _df = pl.read_csv(csv_path + f'.cache.exp.csv')
else:
    csv_path =   rf"~/quant/dragon/{suffix}.csv.{n_bar}m.csv"
    _df = pl.read_csv(csv_path + f'.cache.exp.csv')

mdl_csv = pd.read_csv(model_pth)
eq = mdl_csv['Equation'].values[i]
y_pred = fn(_df,eq)
y_pred = np.abs(y_pred)
y_pred = np.nan_to_num(y_pred, nan=0.0, posinf=0.0, neginf=0.0)
print("Equation ", i, ":", eq)

# unit checking, try to *100 for numerical columns of _df, then compare  fn(_df*100,equations[i]) and y_pred
numerical_cols = _df.select(pl.selectors.by_dtype(pl.NUMERIC_DTYPES)).columns 
_df[numerical_cols] = _df[numerical_cols] * 100
print('Is equation has no unit: ', np.allclose(y_pred,np.nan_to_num(np.abs(fn(_df , eq)), nan=0.0, posinf=0.0, neginf=0.0)))
del _df

"""
Examine Factor  Normality
"""

os.chdir(project_pth)
os.makedirs('./picture',exist_ok=1)
os.makedirs('./temp',exist_ok=1)

df = pl.read_csv(csv_path).to_pandas()
df['factor'] = y_pred

df.rename(columns={'datetime':'eob'})[['eob', 'factor']].rename({'factor': part}).to_csv(
    f'./{suffix}_factor_{part}.csv'
)

def split(arr,freq=30,thred=0):
    n = len(arr)
    arr = np.abs(arr)
    thred = np.nanmean(arr[:180000]) * freq 
    print("thread eqalus: ", thred)
    # 构造 idx_list
    idx_list = []
    cumsum = 0.0
    thred_tmp = thred
    for i in range(n):
        cumsum += arr[i]
        if i == 180000:
            thred_tmp = (thred_tmp * (i-1) + arr[i-1]*(1)) / i
        if cumsum > thred or i == n:
            idx_list.append(i)  # 记录索引
            cumsum = 0.0
    return idx_list

idx_lst = split(y_pred, freq=freq)

df_new_axis = df.iloc[idx_lst]
df_new_axis.reset_index(inplace=True)
df_new_axis.rename(columns = {'index':'hang'},inplace = True)
kde_plot(np.log(df_new_axis['close']),'./picture/kde_new_axis.png')
print('original length:',len(df))
print('original length in {freq} mins :',len(df)//freq)
print('new axis length:',len(df_new_axis))
print('Estimated Frequency of bars: ', int(len(df)/len(df_new_axis)))



"""
Groupping
"""

def split2(arr,freq=30):
    n = len(arr)
    arr = np.abs(arr)
    thred = np.nanmean(arr[:180000]) * freq 
    print("thread eqalus: ", thred)
    # 构造 idx_list
    thred_tmp = thred
    g_lst = []
    g = 0
    cumsum = 0.0
    for i in range(n):
        cumsum += arr[i]
        g_lst.append(g)
        if i == 180000:
            thred_tmp = (thred_tmp * (i-1) + arr[i-1]*(1)) / i
        if cumsum > thred or i == n:
            cumsum = 0.0
            g += 1 
    return g_lst

g_lst = split2(df['factor'].values, freq=freq)
df['group'] =g_lst

Your axis file is in 20250426_100441_OMBx2l


FileNotFoundError: No such file or directory (os error 2): /Users/apple/quant/dragon/RB99_1m.csv.cache.exp.csv

In [1]:
from numba import njit
@njit
def pearson_corr(a,b):
    return np.corrcoef(a,b)[0,1]

@njit 
def rank(x):
    return np.argsort(np.argsort(x))   

@njit
def rank_corr(a,b):
    a = rank(a)
    b = rank(b)
    return pearson_corr(a,b)

@njit(cache=True)
def value_at_risk(a, alpha=0.05):
    a = (a - np.mean(a)) / np.std(a)
    var = np.percentile(a, alpha * 100)
    return var

@njit(cache=True)
def skewness(a):
    a = (a - np.mean(a)) / np.std(a)
    skew = np.mean((a - np.mean(a)) ** 3) / np.std(a) ** 3
    return skew

@njit(cache=True)
def kurtosis(a):
    a = (a - np.mean(a)) / np.std(a)
    kurt = np.mean((a - np.mean(a)) ** 4) / np.std(a) ** 4
    return kurt

@njit(cache=True)
def momentum_5(a):
    a = (a - np.mean(a)) / np.std(a)
    momentum_5 = np.mean((a-np.mean(a))**5) / np.std(a)**5
    return momentum_5


In [ ]:
"""
Alpha#1: (rank(Ts_ArgMax(SignedPower(((returns < 0) ? stddev(returns, 20) : close), 2.), 5)) - 0.5) 
Alpha#2: (-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6)) 
Alpha#3: (-1 * correlation(rank(open), rank(volume), 10)) 
Alpha#4: (-1 * Ts_Rank(rank(low), 9)) 
Alpha#5: (rank((open - (sum(vwap, 10) / 10))) * (-1 * abs(rank((close - vwap))))) 
Alpha#6: (-1 * correlation(open, volume, 10)) 
Alpha#7: ((adv20 < volume) ? ((-1 * ts_rank(abs(delta(close, 7)), 60)) * sign(delta(close, 7))) : (-1 * 1)) 
Alpha#8: (-1 * rank(((sum(open, 5) * sum(returns, 5)) - delay((sum(open, 5) * sum(returns, 5)), 10)))) 
Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
Alpha#10: rank(((0 < ts_min(delta(close, 1), 4)) ? delta(close, 1) : ((ts_max(delta(close, 1), 4) < 0) ? delta(close, 1) : (-1 * delta(close, 1))))) 
Alpha#11: ((rank(ts_max((vwap - close), 3)) + rank(ts_min((vwap - close), 3))) * rank(delta(volume, 3))) 
Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1))) 
Alpha#13: (-1 * rank(covariance(rank(close), rank(volume), 5))) 
Alpha#14: ((-1 * rank(delta(returns, 3))) * correlation(open, volume, 10)) 
Alpha#15: (-1 * sum(rank(correlation(rank(high), rank(volume), 3)), 3)) 
Alpha#16: (-1 * rank(covariance(rank(high), rank(volume), 5))) 
Alpha#17: (((-1 * rank(ts_rank(close, 10))) * rank(delta(delta(close, 1), 1))) * rank(ts_rank((volume / adv20), 5))) 
Alpha#18: (-1 * rank(((stddev(abs((close - open)), 5) + (close - open)) + correlation(close, open, 10)))) 
Alpha#19: ((-1 * sign(((close - delay(close, 7)) + delta(close, 7)))) * (1 + rank((1 + sum(returns, 250))))) 
Alpha#20: (((-1 * rank((open - delay(high, 1)))) * rank((open - delay(close, 1)))) * rank((open - delay(low, 1)))) 
Alpha#21: ((((sum(close, 8) / 8) + stddev(close, 8)) < (sum(close, 2) / 2)) ? (-1 * 1) : (((sum(close, 2) / 2) < ((sum(close, 8) / 8) - stddev(close, 8))) ? 1 : (((1 < (volume / adv20)) || ((volume / adv20) == 1)) ? 1 : (-1 * 1)))) 
Alpha#22: (-1 * (delta(correlation(high, volume, 5), 5) * rank(stddev(close, 20)))) 
Alpha#23: (((sum(high, 20) / 20) < high) ? (-1 * delta(high, 2)) : 0) 
Alpha#24: ((((delta((sum(close, 100) / 100), 100) / delay(close, 100)) < 0.05) || ((delta((sum(close, 100) / 100), 100) / delay(close, 100)) == 0.05)) ? (-1 * (close - ts_min(close, 100))) : (-1 * delta(close, 3))) 
Alpha#25: rank(((((-1 * returns) * adv20) * vwap) * (high - close))) 
Alpha#26: (-1 * ts_max(correlation(ts_rank(volume, 5), ts_rank(high, 5), 5), 3)) 
Alpha#27: ((0.5 < rank((sum(correlation(rank(volume), rank(vwap), 6), 2) / 2.0))) ? (-1 * 1) : 1) 
Alpha#28: scale(((correlation(adv20, low, 5) + ((high + low) / 2)) - close)) 
Alpha#29: (min(product(rank(rank(scale(log(sum(ts_min(rank(rank((-1 * rank(delta((close - 1), 5))))), 2), 1))))), 1), 5) + ts_rank(delay((-1 * returns), 6), 5)) 
Alpha#30: (((1.0 - rank(((sign((close - delay(close, 1))) + sign((delay(close, 1) - delay(close, 2)))) + sign((delay(close, 2) - delay(close, 3)))))) * sum(volume, 5)) / sum(volume, 20)) 
Alpha#31: ((rank(rank(rank(decay_linear((-1 * rank(rank(delta(close, 10)))), 10)))) + rank((-1 * delta(close, 3)))) + sign(scale(correlation(adv20, low, 12)))) 
Alpha#32: (scale(((sum(close, 7) / 7) - close)) + (20 * scale(correlation(vwap, delay(close, 5), 230)))) 
Alpha#33: rank((-1 * ((1 - (open / close))^1))) 
Alpha#34: rank(((1 - rank((stddev(returns, 2) / stddev(returns, 5)))) + (1 - rank(delta(close, 1)))))
Alpha#35: ((Ts_Rank(volume, 32) * (1 - Ts_Rank(((close + high) - low), 16))) * (1 - Ts_Rank(returns, 32))) 
Alpha#36: (((((2.21 * rank(correlation((close - open), delay(volume, 1), 15))) + (0.7 * rank((open - close)))) + (0.73 * rank(Ts_Rank(delay((-1 * returns), 6), 5)))) + rank(abs(correlation(vwap, adv20, 6)))) + (0.6 * rank((((sum(close, 200) / 200) - open) * (close - open))))) 
Alpha#37: (rank(correlation(delay((open - close), 1), close, 200)) + rank((open - close))) 
Alpha#38: ((-1 * rank(Ts_Rank(close, 10))) * rank((close / open))) 
Alpha#39: ((-1 * rank((delta(close, 7) * (1 - rank(decay_linear((volume / adv20), 9)))))) * (1 + rank(sum(returns, 250)))) 
Alpha#40: ((-1 * rank(stddev(high, 10))) * correlation(high, volume, 10)) 
Alpha#41: (((high * low)^0.5) - vwap) 
Alpha#42: (rank((vwap - close)) / rank((vwap + close))) 
Alpha#43: (ts_rank((volume / adv20), 20) * ts_rank((-1 * delta(close, 7)), 8))
Alpha#44: (-1 * correlation(high, rank(volume), 5)) 
Alpha#45: (-1 * ((rank((sum(delay(close, 5), 20) / 20)) * correlation(close, volume, 2)) * rank(correlation(sum(close, 5), sum(close, 20), 2)))) 
Alpha#46: ((0.25 < (((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10))) ? (-1 * 1) : (((((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10)) < 0) ? 1 : ((-1 * 1) * (close - delay(close, 1))))) 
Alpha#47: ((((rank((1 / close)) * volume) / adv20) * ((high * rank((high - close))) / (sum(high, 5) / 5))) - rank((vwap - delay(vwap, 5)))) 
Alpha#48: (indneutralize(((correlation(delta(close, 1), delta(delay(close, 1), 1), 250) * delta(close, 1)) / close), IndClass.subindustry) / sum(((delta(close, 1) / delay(close, 1))^2), 250)) 
Alpha#49: (((((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10)) < (-1 * 0.1)) ? 1 : ((-1 * 1) * (close - delay(close, 1)))) 
Alpha#50: (-1 * ts_max(rank(correlation(rank(volume), rank(vwap), 5)), 5)) 
Alpha#51: (((((delay(close, 20) - delay(close, 10)) / 10) - ((delay(close, 10) - close) / 10)) < (-1 * 0.05)) ? 1 : ((-1 * 1) * (close - delay(close, 1)))) 
Alpha#52: ((((-1 * ts_min(low, 5)) + delay(ts_min(low, 5), 5)) * rank(((sum(returns, 240) - sum(returns, 20)) / 220))) * ts_rank(volume, 5)) 
Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low)), 9)) 
Alpha#54: ((-1 * ((low - close) * (open^5))) / ((low - high) * (close^5))) 
Alpha#55: (-1 * correlation(rank(((close - ts_min(low, 12)) / (ts_max(high, 12) - ts_min(low, 12)))), rank(volume), 6)) 
Alpha#56: (0 - (1 * (rank((sum(returns, 10) / sum(sum(returns, 2), 3))) * rank((returns * cap))))) 
Alpha#57: (0 - (1 * ((close - vwap) / decay_linear(rank(ts_argmax(close, 30)), 2)))) 
Alpha#58: (-1 * Ts_Rank(decay_linear(correlation(IndNeutralize(vwap, IndClass.sector), volume, 3.92795), 7.89291), 5.50322)) 
Alpha#59: (-1 * Ts_Rank(decay_linear(correlation(IndNeutralize(((vwap * 0.728317) + (vwap * (1 - 0.728317))), IndClass.industry), volume, 4.25197), 16.2289), 8.19648)) 
Alpha#60: (0 - (1 * ((2 * scale(rank(((((close - low) - (high - close)) / (high - low)) * volume)))) - scale(rank(ts_argmax(close, 10))))))
Alpha#61: (rank((vwap - ts_min(vwap, 16.1219))) < rank(correlation(vwap, adv180, 17.9282))) 
Alpha#62: ((rank(correlation(vwap, sum(adv20, 22.4101), 9.91009)) < rank(((rank(open) + rank(open)) < (rank(((high + low) / 2)) + rank(high))))) * -1) 
Alpha#63: ((rank(decay_linear(delta(IndNeutralize(close, IndClass.industry), 2.25164), 8.22237)) - rank(decay_linear(correlation(((vwap * 0.318108) + (open * (1 - 0.318108))), sum(adv180, 37.2467), 13.557), 12.2883))) * -1) 
Alpha#64: ((rank(correlation(sum(((open * 0.178404) + (low * (1 - 0.178404))), 12.7054), sum(adv120, 12.7054), 16.6208)) < rank(delta(((((high + low) / 2) * 0.178404) + (vwap * (1 - 0.178404))), 3.69741))) * -1) 
Alpha#65: ((rank(correlation(((open * 0.00817205) + (vwap * (1 - 0.00817205))), sum(adv60, 8.6911), 6.40374)) < rank((open - ts_min(open, 13.635)))) * -1) 
Alpha#66: ((rank(decay_linear(delta(vwap, 3.51013), 7.23052)) + Ts_Rank(decay_linear(((((low * 0.96633) + (low * (1 - 0.96633))) - vwap) / (open - ((high + low) / 2))), 11.4157), 6.72611)) * -1) 
Alpha#67: ((rank((high - ts_min(high, 2.14593)))^rank(correlation(IndNeutralize(vwap, IndClass.sector), IndNeutralize(adv20, IndClass.subindustry), 6.02936))) * -1) 
Alpha#68: ((Ts_Rank(correlation(rank(high), rank(adv15), 8.91644), 13.9333) < rank(delta(((close * 0.518371) + (low * (1 - 0.518371))), 1.06157))) * -1) 
Alpha#69: ((rank(ts_max(delta(IndNeutralize(vwap, IndClass.industry), 2.72412), 4.79344))^Ts_Rank(correlation(((close * 0.490655) + (vwap * (1 - 0.490655))), adv20, 4.92416), 9.0615)) * -1) 
Alpha#70: ((rank(delta(vwap, 1.29456))^Ts_Rank(correlation(IndNeutralize(close, IndClass.industry), adv50, 17.8256), 17.9171)) * -1) 
Alpha#71: max(Ts_Rank(decay_linear(correlation(Ts_Rank(close, 3.43976), Ts_Rank(adv180, 12.0647), 18.0175), 4.20501), 15.6948), Ts_Rank(decay_linear((rank(((low + open) - (vwap + vwap)))^2), 16.4662), 4.4388)) 
Alpha#72: (rank(decay_linear(correlation(((high + low) / 2), adv40, 8.93345), 10.1519)) / rank(decay_linear(correlation(Ts_Rank(vwap, 3.72469), Ts_Rank(volume, 18.5188), 6.86671), 2.95011))) 
Alpha#73: (max(rank(decay_linear(delta(vwap, 4.72775), 2.91864)), Ts_Rank(decay_linear(((delta(((open * 0.147155) + (low * (1 - 0.147155))), 2.03608) / ((open * 0.147155) + (low * (1 - 0.147155)))) * -1), 3.33829), 16.7411)) * -1) 
Alpha#74: ((rank(correlation(close, sum(adv30, 37.4843), 15.1365)) < rank(correlation(rank(((high * 0.0261661) + (vwap * (1 - 0.0261661)))), rank(volume), 11.4791))) * -1) 
Alpha#75: (rank(correlation(vwap, volume, 4.24304)) < rank(correlation(rank(low), rank(adv50), 12.4413))) 
Alpha#76: (max(rank(decay_linear(delta(vwap, 1.24383), 11.8259)), Ts_Rank(decay_linear(Ts_Rank(correlation(IndNeutralize(low, IndClass.sector), adv81, 8.14941), 19.569), 17.1543), 19.383)) * -1) 
Alpha#77: min(rank(decay_linear(((((high + low) / 2) + high) - (vwap + high)), 20.0451)), rank(decay_linear(correlation(((high + low) / 2), adv40, 3.1614), 5.64125))) 
Alpha#78: (rank(correlation(sum(((low * 0.352233) + (vwap * (1 - 0.352233))), 19.7428), sum(adv40, 19.7428), 6.83313))^rank(correlation(rank(vwap), rank(volume), 5.77492))) 
Alpha#79: (rank(delta(IndNeutralize(((close * 0.60733) + (open * (1 - 0.60733))), IndClass.sector), 1.23438)) < rank(correlation(Ts_Rank(vwap, 3.60973), Ts_Rank(adv150, 9.18637), 14.6644))) 
Alpha#80: ((rank(Sign(delta(IndNeutralize(((open * 0.868128) + (high * (1 - 0.868128))), IndClass.industry), 4.04545)))^Ts_Rank(correlation(high, adv10, 5.11456), 5.53756)) * -1) 
Alpha#81: ((rank(Log(product(rank((rank(correlation(vwap, sum(adv10, 49.6054), 8.47743))^4)), 14.9655))) < rank(correlation(rank(vwap), rank(volume), 5.07914))) * -1) 
Alpha#82: (min(rank(decay_linear(delta(open, 1.46063), 14.8717)), Ts_Rank(decay_linear(correlation(IndNeutralize(volume, IndClass.sector), ((open * 0.634196) + (open * (1 - 0.634196))), 17.4842), 6.92131), 13.4283)) * -1) 
Alpha#83: ((rank(delay(((high - low) / (sum(close, 5) / 5)), 2)) * rank(rank(volume))) / (((high - low) / (sum(close, 5) / 5)) / (vwap - close))) 
Alpha#84: SignedPower(Ts_Rank((vwap - ts_max(vwap, 15.3217)), 20.7127), delta(close, 4.96796)) 
Alpha#85: (rank(correlation(((high * 0.876703) + (close * (1 - 0.876703))), adv30, 9.61331))^rank(correlation(Ts_Rank(((high + low) / 2), 3.70596), Ts_Rank(volume, 10.1595), 7.11408))) 
Alpha#86: ((Ts_Rank(correlation(close, sum(adv20, 14.7444), 6.00049), 20.4195) < rank(((open + close) - (vwap + open)))) * -1) 
Alpha#87: (max(rank(decay_linear(delta(((close * 0.369701) + (vwap * (1 - 0.369701))), 1.91233), 2.65461)), Ts_Rank(decay_linear(abs(correlation(IndNeutralize(adv81, IndClass.industry), close, 13.4132)), 4.89768), 14.4535)) * -1) 
Alpha#88: min(rank(decay_linear(((rank(open) + rank(low)) - (rank(high) + rank(close))), 8.06882)), Ts_Rank(decay_linear(correlation(Ts_Rank(close, 8.44728), Ts_Rank(adv60, 20.6966), 8.01266), 6.65053), 2.61957)) 
Alpha#89: (Ts_Rank(decay_linear(correlation(((low * 0.967285) + (low * (1 - 0.967285))), adv10, 6.94279), 5.51607), 3.79744) - Ts_Rank(decay_linear(delta(IndNeutralize(vwap, IndClass.industry), 3.48158), 10.1466), 15.3012)) 
Alpha#90: ((rank((close - ts_max(close, 4.66719)))^Ts_Rank(correlation(IndNeutralize(adv40, IndClass.subindustry), low, 5.38375), 3.21856)) * -1) 
Alpha#91: ((Ts_Rank(decay_linear(decay_linear(correlation(IndNeutralize(close, IndClass.industry), volume, 9.74928), 16.398), 3.83219), 4.8667) - rank(decay_linear(correlation(vwap, adv30, 4.01303), 2.6809))) * -1) 
Alpha#92: min(Ts_Rank(decay_linear(((((high + low) / 2) + close) < (low + open)), 14.7221), 18.8683), Ts_Rank(decay_linear(correlation(rank(low), rank(adv30), 7.58555), 6.94024), 6.80584)) 
Alpha#93: (Ts_Rank(decay_linear(correlation(IndNeutralize(vwap, IndClass.industry), adv81, 17.4193), 19.848), 7.54455) / rank(decay_linear(delta(((close * 0.524434) + (vwap * (1 - 0.524434))), 2.77377), 16.2664))) 
Alpha#94: ((rank((vwap - ts_min(vwap, 11.5783)))^Ts_Rank(correlation(Ts_Rank(vwap, 19.6462), Ts_Rank(adv60, 4.02992), 18.0926), 2.70756)) * -1) 
Alpha#95: (rank((open - ts_min(open, 12.4105))) < Ts_Rank((rank(correlation(sum(((high + low) / 2), 19.1351), sum(adv40, 19.1351), 12.8742))^5), 11.7584)) 
Alpha#96: (max(Ts_Rank(decay_linear(correlation(rank(vwap), rank(volume), 3.83878), 4.16783), 8.38151), Ts_Rank(decay_linear(Ts_ArgMax(correlation(Ts_Rank(close, 7.45404), Ts_Rank(adv60, 4.13242), 3.65459), 12.6556), 14.0365), 13.4143)) * -1) 
Alpha#97: ((rank(decay_linear(delta(IndNeutralize(((low * 0.721001) + (vwap * (1 - 0.721001))), IndClass.industry), 3.3705), 20.4523)) - Ts_Rank(decay_linear(Ts_Rank(correlation(Ts_Rank(low, 7.87871), Ts_Rank(adv60, 17.255), 4.97547), 18.5925), 15.7152), 6.71659)) * -1) 
Alpha#98: (rank(decay_linear(correlation(vwap, sum(adv5, 26.4719), 4.58418), 7.18088)) - rank(decay_linear(Ts_Rank(Ts_ArgMin(correlation(rank(open), rank(adv15), 20.8187), 8.62571), 6.95668), 8.07206))) 
Alpha#99: ((rank(correlation(sum(((high + low) / 2), 19.8975), sum(adv60, 19.8975), 8.8136)) < rank(correlation(low, volume, 6.28259))) * -1) 
Alpha#100: (0 - (1 * (((1.5 * scale(indneutralize(indneutralize(rank(((((close - low) - (high - close)) / (high - low)) * volume)), IndClass.subindustry), IndClass.subindustry))) - scale(indneutralize((correlation(close, rank(adv20), 5) - rank(ts_argmin(close, 30))), IndClass.subindustry))) * (volume / adv20)))) 
Alpha#101: ((close - open) / ((high - low) + .001)) 

"""

In [53]:
from functions import *
import numpy as np
import pandas as pd

w = 10

x = np.array(np.random.randn(1000))
y = np.array(np.random.randn(1000))

x = reshape(x, w)
y = reshape(y, w)

df = pd.DataFrame(
    {
        'open':np.random.randn(10000)+np.random.randn(10000)*2,
        'high':np.random.randn(10000),
        'low':np.random.randn(10000),
        'close':np.random.randn(10000),
        'volume':np.random.randn(10000),
        'adv20':np.random.randn(10000),
    }
)
for x in df.columns:
    df[x] = df[x] - df[x].min() + 1e-5


np_df = df.values 


def alpha_3(arrs):
    _high = arrs[:, 1]
    _volume = arrs[:, 4]

    return (-1 * _high / _volume)

def alpha_4(arrs):
    _high = arrs[:, 1]
    _bot = np.minimum(arrs[:, 2], arrs[:, 3])
    h2bot = _high / _bot

    return max_(h2bot, 1.0)


def alpha_5(arrs):
    # rank_corr of volume and close 
    _close = arrs[:, 3]
    _volume = arrs[:, 4]
    return rank_corr(_close, _volume)


def inside_bar_lifting_speed(arrs,i):
    _price = arrs[:, i]
    return (_price[-1]/_price[0]-1)/len(arrs)

def inside_bar_length(arrs):
    return len(arrs) - 1

def inside_bar_corr_price_volume(arrs, i, j):
    _price = arrs[:, i]
    _volume = arrs[:, j]
    return rank_corr(_price, _volume)

def inside_bar_skew(arrs,i):
    _price = arrs[:,i]
    _ret = log_ret(_price)
    return skewness(_ret)

def inside_bar_kurt(arrs,i):
    _price = arrs[:,i]
    _ret = log_ret(_price)
    return kurtosis(_ret)

def inside_bar_momentum_n(arrs,i,n=5):
    _price = arrs[:,i]
    _ret = log_ret(_price)
    return momentum(_ret, n)

def inside_bar_zero_cross(arrs,i):
    _price = arrs[:,i]
    _price = _price - np.mean(_price)
    return ant.num_zerocross(_price)

def inside_bar_lpcc(arrs, i):
    _price = arrs[:,i].flatten()
    # print(len(_price))
    return lpcc(_price,12, 11)

def inside_bar_sample_entropy(arrs,i, m=2, r_ratio=1, use_std=True):
    _price = arrs[:,i].flatten()
    _price = reshape(_price,30)
    return dynamic_sample_entropy_numba(_price, m=m, r_ratio=r_ratio, use_std=use_std)

def inside_bar_spectral_entropy(arrs,i):
    _price = arrs[:,i].flatten()
    return ant.spectral_entropy(_price, sf=100, method='welch', normalize=True)

def inside_bar_ret_auto_corr(arrs,i):
    _price = arrs[:,i].flatten()
    _ret = log_ret(_price)
    return corr(_ret[:-1], _ret[1:])

def inside_bar_p_auto_corr(arrs,i):
    _price = arrs[:,i].flatten()
    return corr(_price[:-1], _price[1:])

def inside_bar_ret_max(arrs,i):
    _price = arrs[:,i].flatten()
    _ret = log_ret(_price)
    return max(_ret)

def inside_bar_ret_min(arrs,i):
    _price = arrs[:,i].flatten()
    _ret = log_ret(_price)
    return min(_ret)

def inside_bar_petrosian_fd(arrs,i):
    _price = arrs[:,i].flatten()
    return ant.petrosian_fd(_price)

def inside_bar_katz_fd(arrs,i):
    _price = arrs[:,i].flatten()
    return ant.katz_fd(_price)

def inside_bar_higuchi_fd(arrs,i):
    _price = arrs[:,i].flatten()
    return ant.higuchi_fd(_price)

def inside_bar_dfa(arrs,i):
    _price = arrs[:,i].flatten()
    return ant.detrended_fluctuation(_price)

In [55]:
rolling_apply_matrix(inside_bar_higuchi_fd, np_df,100, 0)

array([       nan,        nan,        nan, ..., 2.01788075, 2.02544797,
       2.02734399])